In [1]:
# Now I need to recalculate lake metadata (weights ect) in the same manner as the catchments
import ECCO_functions_v2 as ECCO

import pandas as pd
import osgeo.ogr
import sys, time, os, json, glob
import numpy as np
import pyproj
import h5py
import csv  # <<-- dont forget to add this ti function file
import random

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def MT_Gen_SWeights(nc_path):
    '''
    Purpose:          
    This program Generates metadata files used to speed up the final runs.
    This was forthe lakes (not catchments), and created a metadata text
    file, to be used as a mini-database in pandas.
    '''
        
    clim_dat,rlat,rlon,time,metadata,txtfname = ECCO.Read_CORDEX_V2(nc_path)
    vname, m1, m2, dexp, m3, m4, m5, m6, drange_orignial = metadata 
 
    var_type = clim_dat.standard_name       # What kind of CORDEX data?
    dat_loaded = clim_dat[0,:,:]            # Load CORDEX data into RAM
    print np.shape(dat_loaded)
    rlat_loaded = rlat[:]
    rlon_loaded = rlon[:]

    lake_file = 'Lakes/ecco-biwa_lakes_v.0.2.shp'
    
    thefilename = 'lake_weights'
    FILE= 'Lakes/Weights/' + thefilename +'.h5'                # Set up HDF5 file output
    if os.path.isfile(FILE):
        #print 'HDF5 File already exists. Leaving loop so you dont clobber it by accident.'
        #print 'To run this function, decide manually if you want to remove it or not.'
        #return
        print 'hdf weights file exists, removing it...'
        os.remove(FILE)
    else:
        print 'Creating file: ',FILE
    fweights = h5py.File(FILE,'w')
    
    #orog = ECCO.Height_CORDEX()
    ShapeData = osgeo.ogr.Open(lake_file)
    TheLayer = ShapeData.GetLayer(iLayer=0)
    dolakes=range(TheLayer.GetFeatureCount())
    #alist = []
    #blist = []
    for num in dolakes[0:10]:
        feature1 = TheLayer.GetFeature(num) 
        lake_feature = feature1.ExportToJson(as_object=True)
        lake_cart = ECCO.Path_LkIsl_ShpFile(lake_feature['geometry']['coordinates']) 
        EB_id = lake_feature['properties']['EBhex'][2:]
        lake_altitude=lake_feature['properties']['vfp_mean']

        lake_rprj = ECCO.Path_Reproj(lake_cart,False)

        sub_clim,sub_rlat,sub_rlon = ECCO.TrimToLake(lake_rprj,dat_loaded,rlat_loaded,
                                                        rlon_loaded,off = 3, show = False) 
        weight_mask = ECCO.Pixel_Weights(lake_rprj,sub_clim,sub_rlat,sub_rlon)


        pix_truth = (weight_mask > 0.0)    # Count how many times the weight mask is
        pxnum = len(weight_mask[pix_truth])  #  above 0.0 (i.e. how many pixels of data are needed)
    
        etime = 1.0  # Nothing, can be removed (in the meta too, but be careful to change the later programs)

        #print EB_id,' Weight mask sum:',np.sum(weight_mask)
        #alist.append(EB_id)
        #blist.append(np.sum(weight_mask))
        
        ypix = -99
        xpix = -99
        if pxnum == 1:
            xxx,yyy = ECCO.Get_LatLonLim(lake_rprj.vertices)  # Find upp./low.lake lims.
            ypix = (ECCO.Closest(rlat,yyy[0]))                # For lakes of one pixel  
            xpix = (ECCO.Closest(rlon,xxx[0]))
        if pxnum < 1:
            pxnum = 1  # Small bug where it thinks lakes dont exist, no biggy...
        if pxnum > 1:
            ECCO.Write_HDF_weights(fw=fweights,EB_id=EB_id,weights=weight_mask) 
        
        print num,EB_id[2:], ECCO.Area_Lake_and_Islands(lake_cart),pxnum,etime,ypix,xpix
        
    print 'End of function'
    return #alist, blist

In [5]:
#ncfile= '/uio/kant/geo-metos-u1/blaken/datadisk/ECCO/CORDEX/Data_CORDEX/tas_EUR-11_ICHEC-EC-EARTH_historical_r1i1p1_KNMI-RACMO22E_v1_day_19660101-19701231.nc'
ncfile ='CORDEX/tas_EUR-11_ICHEC-EC-EARTH_rcp45_r1i1p1_KNMI-RACMO22E_v1_day_20960101-21001231.nc'

namelist,sumlist = MT_Gen_SWeights(nc_path=ncfile)

(412, 424)
hdf weights file exists, removing it...
